**Importing libraries**

In [1]:
import numpy as np 
import pandas as pd 
import multiprocessing as multi
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt
import os
from IPython.display import clear_output
from scipy.optimize import curve_fit

**Capturing name of the PC**

In [2]:
%%capture PC_name 
!hostname

**Setting notebook parameters**

In [3]:
#setting dots per inch for images
dpi = 300

#setting number of processors for multiprocessing
PC_name_str = PC_name.stdout[0:-2]

if PC_name_str == 'science12':
    cores = 42        
else:
    cores = multi.cpu_count()

print(PC_name_str)
print(cores)

science12
42


**Defining constants**

In [4]:
omega_m = 0.272            #matter density paratmeter from Komatsu et al. (2011) 
omega_l = 1 - omega_m      #vacuum density paratmeter assuming flat universe
H_o = 70.4                 #Hubble constant in km s^−1 Mpc^−1 from Komatsu et al. (2011) 
h = 0.704
f = omega_m**0.545         #linear velocity growth rate from Tanimura et al. (2020)


**Defining Functions**

In [5]:
def H(z):                           #hubble parameter (in km s^-1 Mpc^-1) using Eq. 4.33 in Peter's book
    return np.sqrt( H_o**2 * ( (1+z)**3 * omega_m + omega_l ) ) 


**Reading galaxies dataset**

In [6]:
%%time
z = 0.42372720
# data_address = '../input/magneticumsnap027z042-massfiltered/'
data_address = 'Data/'
df_gal = pd.read_csv(data_address + 'massive_galaxies.csv')      #massive_galaxies.csv contains galaxies with mass greater than 1.8 ×10^{11} h^{−1} M_sun as done by Tanimura et al. (2020)
df_gal.describe()


CPU times: user 61.2 ms, sys: 10.3 ms, total: 71.5 ms
Wall time: 69.5 ms


,x[kpc/h],y[kpc/h],z[kpc/h],m[Msol/h]
count,93097.000000,93097.000000,93097.000000,9.309700e+04
mean,325576.133087,322500.028738,320613.762863,3.993206e+11
std,184863.273960,182799.587421,183249.915097,3.995129e+11
min,1.864417,10.164207,0.260974,1.800010e+11
25%,163866.950000,166201.380000,162514.560000,2.161480e+11
50%,332449.620000,329260.160000,319925.780000,2.784900e+11
75%,483949.000000,475466.120000,478698.410000,4.192930e+11
max,639996.190000,639980.940000,639993.120000,1.077460e+13


**Reading clusters dataset**

In [7]:
df_clusters_orig = pd.read_csv(data_address + 'massive_clusters.csv', 
                          usecols = ['x[kpc/h]', 'y[kpc/h]', 'z[kpc/h]', 'm500c[Msol/h]', 'vx[km/s]', 'vy[km/s]', 'vz[km/s]'    ])  #massive_clusters.csv contains clusters with M_500c greater than 10^13.5 h^{-1} M_sun as done by Tanimura et al. (2020)

df_clusters_orig.describe()

,x[kpc/h],y[kpc/h],z[kpc/h],m500c[Msol/h],vx[km/s],vy[km/s],vz[km/s]
count,6080.000000,6080.000000,6080.000000,6.080000e+03,6080.000000,6080.000000,6080.000000
mean,326416.401838,322055.402286,321308.240244,6.356245e+13,2.109102,-1.647297,-4.296055
std,186021.550694,182650.899538,183665.854310,4.851698e+13,316.064539,306.226537,286.454388
min,19.163288,152.653880,87.370949,3.162340e+13,-1179.530000,-1163.520000,-1104.790000
25%,160978.575000,164793.430000,161863.925000,3.795035e+13,-200.229000,-205.139250,-190.806000
50%,338884.565000,333274.345000,322110.075000,4.801630e+13,-3.619450,1.969450,-8.055475
75%,486227.747500,473556.765000,478893.610000,6.945502e+13,206.816750,207.755750,181.532000
max,639887.560000,639933.380000,639848.380000,7.438200e+14,1197.640000,1164.960000,1201.260000


**Removing clusters at edges of the simulation box**

In [8]:
def edge_clus_remover(clus_cube_size):
    
    low_bound = clus_cube_size//2
    upp_bound = 640000 - (clus_cube_size//2)
    
    df_clusters = df_clusters_orig[(df_clusters_orig['x[kpc/h]'] > low_bound) & 
                                   (df_clusters_orig['x[kpc/h]'] < upp_bound) & 
                                   (df_clusters_orig['y[kpc/h]'] > low_bound) & 
                                   (df_clusters_orig['y[kpc/h]'] < upp_bound) & 
                                   (df_clusters_orig['z[kpc/h]'] > low_bound) & 
                                   (df_clusters_orig['z[kpc/h]'] < upp_bound)]
    
    return df_clusters

**Specifying prefactors for Eq. 1 of Tanimura et al. (2020)**

In [9]:
a = 1/(1+z)
H(z)
print(H(z))

pre_fac = (f * a * H(z) / (4 * np.pi))           #in km s^−1 Mpc^−1 
pre_fac

86.5938062370014


2.380620866668027

**Calculating mean density of the simulation box for Eq. 1**

In [10]:
def delta_gal_mean_func(cell_size):
    
    df_gal_mean = df_gal.copy()

    df_gal_mean['x[kpc/h]'] = df_gal_mean['x[kpc/h]'] / cell_size
    df_gal_mean['y[kpc/h]'] = df_gal_mean['y[kpc/h]'] / cell_size
    df_gal_mean['z[kpc/h]'] = df_gal_mean['z[kpc/h]'] / cell_size

    #making tuples, converting tuples to cell coordinates
    df_gal_mean["cell"] = list(zip(df_gal_mean['x[kpc/h]'].astype(int), df_gal_mean['y[kpc/h]'].astype(int), df_gal_mean['z[kpc/h]'].astype(int)))

    #array to store number of galaxies in the cells
    gals_in_cell = np.zeros((640000//cell_size, 640000//cell_size, 640000//cell_size))
    
    #counting number of galaxies in the cells
    for cell in df_gal_mean["cell"]:
        x, y, z = cell
        gals_in_cell[x, y, z] += 1

    delta_gal_mean = np.mean(gals_in_cell)
    
    return delta_gal_mean

**Calculating Overdensity field for Eq. 1**

In [11]:
b = 2                                #bias for LOWZ & CMASS galaxies as taken by Tanimura et al. 2020

def overdensity_field_calc(clus_x, clus_y, clus_z, delta_gal_mean, cell_size, sigma_in_pix, clus_cube_size):

    #converting strings into floats
    clus_x = float(clus_x); clus_y = float(clus_y); clus_z = float(clus_z)
    
    #filtering galaxies in a cube of 240,000 h^-1 kpc around the given cluster
    df_gal_select = df_gal[(df_gal['x[kpc/h]'] > (clus_x - clus_cube_size//2)) & 
                           (df_gal['x[kpc/h]'] < (clus_x + clus_cube_size//2)) & 
                           (df_gal['y[kpc/h]'] > (clus_y - clus_cube_size//2)) & 
                           (df_gal['y[kpc/h]'] < (clus_y + clus_cube_size//2)) & 
                           (df_gal['z[kpc/h]'] > (clus_z - clus_cube_size//2)) & 
                           (df_gal['z[kpc/h]'] < (clus_z + clus_cube_size//2))]
    
    #making copy to extract coordinates of cells containing the galaxies
    df_gal_cube = df_gal_select.copy()
    
    #moving the galxies cube to lie within 0 to 240,000 h^-1 kpc 
    df_gal_cube['x[kpc/h]'] -= (clus_x - clus_cube_size//2)
    df_gal_cube['y[kpc/h]'] -= (clus_y - clus_cube_size//2)
    df_gal_cube['z[kpc/h]'] -= (clus_z - clus_cube_size//2)

    #dividing by 5000 (integer-div) so we get cell coordinates
    df_gal_cube['x[kpc/h]'] = df_gal_cube['x[kpc/h]'] / cell_size
    df_gal_cube['y[kpc/h]'] = df_gal_cube['y[kpc/h]'] / cell_size
    df_gal_cube['z[kpc/h]'] = df_gal_cube['z[kpc/h]'] / cell_size
    
    #making tuples, converting tuples to cell coordinates
    df_gal_cube["cell"] = list(zip(df_gal_cube['x[kpc/h]'].astype(int), df_gal_cube['y[kpc/h]'].astype(int), df_gal_cube['z[kpc/h]'].astype(int)))
    
    #array to store number of galaxies in the cells
    gals_in_cell = np.zeros((clus_cube_size//cell_size, clus_cube_size//cell_size, clus_cube_size//cell_size))
    
    #counting number of galaxies in the cells
    for cell in df_gal_cube["cell"]:
        x, y, z = cell
        gals_in_cell[x, y, z] += 1
        
    #Error in gals_in_cell
    gals_in_cell_error = np.sqrt(gals_in_cell)
        
    #determining the overdensity of galaxies    
    delta_gal = (gals_in_cell/delta_gal_mean) - 1
    delta_gal_error = gals_in_cell_error/delta_gal_mean
    
    #smoothing the overdensity of galaxies
    delta_gal_smooth = gaussian_filter(delta_gal, sigma = sigma_in_pix)
        
    #obtaining matter overdensity from galaxies overdensity
    delta_matter = delta_gal_smooth / b
    delta_matter_error = delta_gal_error / b
    
    return delta_matter, delta_matter_error


**Calculating differential, numerator & denominator for Eq. 1**

In [12]:
def vel_terms_calc(cell_size, clus_cube_size):
    
    #calculating the differential in the Eq. 1
    dy_cubed = cell_size**3
    
    #specifing position of the clusters
    Rclus_x = np.zeros((clus_cube_size//cell_size, clus_cube_size//cell_size, clus_cube_size//cell_size))
    Rclus_x[:] = clus_cube_size//2
    Rclus_y = np.zeros((clus_cube_size//cell_size, clus_cube_size//cell_size, clus_cube_size//cell_size))
    Rclus_y[:] = clus_cube_size//2
    Rclus_z = np.zeros((clus_cube_size//cell_size, clus_cube_size//cell_size, clus_cube_size//cell_size))
    Rclus_z[:] = clus_cube_size//2

    #generating meshgrid containing coordinates of the centers of cells
    Rcell_x = np.zeros((clus_cube_size//cell_size, clus_cube_size//cell_size, clus_cube_size//cell_size))
    Rcell_y = np.zeros((clus_cube_size//cell_size, clus_cube_size//cell_size, clus_cube_size//cell_size))
    Rcell_z = np.zeros((clus_cube_size//cell_size, clus_cube_size//cell_size, clus_cube_size//cell_size))

    for i, val in enumerate(range(cell_size//2, clus_cube_size, cell_size)):
        Rcell_x[i,:,:] = val
        Rcell_y[:,i,:] = val
        Rcell_z[:,:,i] = val

    #evaluating the term in the denominator of Eq. 1 of Tanimura et al. 2020
    denom = np.sqrt((Rcell_x - Rclus_x)**2 + (Rcell_y - Rclus_y)**2 + (Rcell_z - Rclus_z)**2)**(3)

    #evaluating the direction term in the numerator of Eq. 1
    numer_x = Rcell_x - Rclus_x
    numer_y = Rcell_y - Rclus_y
    numer_z = Rcell_z - Rclus_z
    
    return (dy_cubed, numer_x, numer_y, numer_z, denom)


**Calculating velocity of clusters according to Eq. 1**

In [13]:
def clus_velocity_calc(clus_x, clus_y, clus_z, delta_gal_mean, cell_size, vel_terms, sigma_in_pix, 
                       clus_cube_size):
    
    delta_matter, delta_matter_error = overdensity_field_calc(clus_x, clus_y, clus_z, delta_gal_mean, cell_size, sigma_in_pix, 
                                          clus_cube_size)
    
    
    dy_cubed, numer_x, numer_y, numer_z, denom = vel_terms
       
    #estimating velocity in x direction
    integrand_x = dy_cubed * delta_matter * (numer_x/(h*1e3)) / denom #in units of Mpc
    integrand_x_error = dy_cubed * delta_matter_error * (numer_x/(h*1e3)) / denom
    
    vx_est = pre_fac * np.sum(integrand_x)
    vx_est_error = pre_fac * np.sqrt(np.sum(integrand_x_error**2))
        
    #estimating velocity in y direction
    integrand_y = dy_cubed * delta_matter * (numer_y/(h*1e3)) / denom #in units of Mpc    
    integrand_y_error = dy_cubed * delta_matter_error * (numer_y/(h*1e3)) / denom
    
    vy_est = pre_fac * np.sum(integrand_y)
    vy_est_error = pre_fac * np.sqrt(np.sum(integrand_y_error**2))
    
    #estimating velocity in z direction
    integrand_z = dy_cubed * delta_matter * (numer_z/(h*1e3)) / denom #in units of Mpc    
    integrand_z_error = dy_cubed * delta_matter_error * (numer_z/(h*1e3)) / denom
    
    vz_est = pre_fac * np.sum(integrand_z)
    vz_est_error = pre_fac * np.sqrt(np.sum(integrand_z_error**2))
    
    return(vx_est, vy_est, vz_est, vx_est_error, vy_est_error, vz_est_error)


**Calculating sigma for smoothing**

In [14]:
def sigma_calc(cell_size):
    
    FWHM = 2000                           #h^-1 kpc, of Gaussian kernel, taken by Tanimura et al. 2020
    FWHM_in_pix = FWHM/cell_size          #in pixel units
    sigma_in_pix = FWHM_in_pix/(2.35482)  #in pixel units
    
    return sigma_in_pix

**Plotting the scatter plots & histograms for assesment of velocity estimates**

In [119]:
def plotting_func(df_clusters_est_err, cell_size, clus_cube_size_actual):
    
    fig = plt.figure(dpi = dpi, figsize = (10,12), facecolor=(1, 1, 1))

    point_size = 4
    cluster_jump = 50
    
    plt.subplot(3,2,1)
    plt.errorbar(df_clusters_est_err['vx[km/s]'][::cluster_jump], df_clusters_est_err['vx_est[km/s]'][::cluster_jump], 
                 yerr = df_clusters_est_err['vx_est_err[km/s]'][::cluster_jump], fmt='.', ecolor='C6')
    plt.xlabel('V$_\mathrm{x, true}$ (km/s)')
    plt.ylabel('V$_\mathrm{x, estimated}$ (km/s)')
    low_lim, high_lim = -2000, 2000
    plt.gca().set_xticks(range(low_lim, high_lim+1, 1000))
    plt.gca().set_yticks(range(low_lim, high_lim, 1000))
    plt.xlim(low_lim, high_lim)
    plt.ylim(low_lim,high_lim)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.title('Vx - scatter plot')
    
    x_pos_text = 0.40
    x_pos_text_hist = 0.65
    text_font_size = 8
    
    plt.text(x_pos_text, 0.15, f'Clusters: {len(df_clusters_est_err)} ({cluster_jump}th plotted)', transform=plt.gca().transAxes, fontsize = text_font_size)
     
    x_range = np.arange(low_lim, high_lim+1, 5)    
    def fit_func(x, m, c):
        return m*x + c
    
    param_x, cov_x = curve_fit(fit_func, df_clusters_est_err['vx[km/s]'], df_clusters_est_err['vx_est[km/s]'], 
                           sigma = df_clusters_est_err['vx_est_err[km/s]'], absolute_sigma=True)
    
    plt.text(x_pos_text, 0.09, f'Slope: {param_x[0].round(2)} $\pm$ {cov_x[0,0].round(2)}', transform=plt.gca().transAxes, fontsize = text_font_size)
    plt.text(x_pos_text, 0.03, f'Zero point: {param_x[1].round(2)} $\pm$ {cov_x[1,1].round(2)} km/s', transform=plt.gca().transAxes, fontsize = text_font_size)
    
    plt.plot(x_range, fit_func(x_range, *param_x), label = 'Linear fit', c = 'C2')
    plt.plot(x_range, x_range, label = '1-1 line', c = 'C1', alpha = 0.25, ls = 'dashed')
    plt.legend(loc = 'upper left')
    
    plt.subplot(3,2,2)
    residuals_x = fit_func(df_clusters_est_err['vx[km/s]'], *param_x) - df_clusters_est_err['vx_est[km/s]']
    plt.hist(residuals_x, bins = 100)
    plt.xlabel('Residual: V$_{\mathrm{x, fit}}$ - V$_{\mathrm{x, est}}$ (km/s)')
    plt.ylabel('Number of clusters')
    x_low, x_high = plt.xlim()
    plt.xlim(x_low, abs(x_low))
    y_low, y_high = plt.ylim()
    plt.ylim(y_low, y_high + 15)
    
    plt.text(0.02, 0.90, r'Mean (V$_{\mathrm{x, est}}$): ' + f"{round(np.mean(df_clusters_est_err['vx_est[km/s]']), 1)} km/s", transform=plt.gca().transAxes, fontsize = text_font_size)
    plt.text(0.02, 0.83, r'SD (V$_{\mathrm{x, est}}$): ' + f"{round(np.std(df_clusters_est_err['vx_est[km/s]']), 1)} km/s", transform=plt.gca().transAxes, fontsize = text_font_size)
    plt.text(0.02, 0.76, r'Mean (Error in V$_{\mathrm{x, est}}$): ' + f"{round(np.mean(df_clusters_est_err['vx_est_err[km/s]']), 1)} km/s", transform=plt.gca().transAxes, fontsize = text_font_size)
    plt.text(0.02, 0.69, r'SD (Error in V$_{\mathrm{x, est}}$): ' + f"{round(np.std(df_clusters_est_err['vx_est_err[km/s]']), 1)} km/s", transform=plt.gca().transAxes, fontsize = text_font_size)
    
    minus_one = {-1}
    plt.text(x_pos_text_hist, 0.90, r'Cell size: {} h$^{}$kpc'.format(cell_size, minus_one), transform=plt.gca().transAxes, fontsize = text_font_size)
    plt.text(x_pos_text_hist, 0.83, r'Cube size: {} h$^{}$kpc'.format(clus_cube_size, minus_one), transform=plt.gca().transAxes, fontsize = text_font_size)
    mean_x = np.mean(residuals_x)
    plt.text(x_pos_text_hist, 0.76, f"Mean (residual): {round(mean_x, 1)} km/s", transform=plt.gca().transAxes, fontsize = text_font_size)
#     SD_x = np.sqrt(np.mean((residuals_x)**2))
    SD_x = np.std(residuals_x)
    plt.text(x_pos_text_hist, 0.69, f"SD (residual): {round(SD_x, 1)} km/s", transform=plt.gca().transAxes, fontsize = text_font_size)
    r_vx = np.corrcoef(df_clusters_est_err['vx[km/s]'], df_clusters_est_err['vx_est[km/s]'])[1,0]
    plt.text(x_pos_text_hist, 0.62, f'Pearson\'s r: {round(r_vx, 2)}', transform=plt.gca().transAxes, fontsize = text_font_size)

    plt.title('Distribution of residuals in V$_{\mathrm{x, est}}$ fitting')




    plt.subplot(3,2,3)
    plt.errorbar(df_clusters_est_err['vy[km/s]'][::cluster_jump], df_clusters_est_err['vy_est[km/s]'][::cluster_jump], 
                 yerr = df_clusters_est_err['vy_est_err[km/s]'][::cluster_jump], fmt='.', ecolor='C6')
    plt.xlabel('V$_\mathrm{y, true}$ (km/s)')
    plt.ylabel('V$_\mathrm{y, estimated}$ (km/s)')
    plt.gca().set_xticks(range(-2000, 2001, 1000))
    plt.gca().set_yticks(range(-2000, 2001, 1000))
    plt.xlim(low_lim, high_lim)
    plt.ylim(-2000,2000)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.title('Vy - scatter plot')
    plt.text(x_pos_text, 0.15, f'Clusters: {len(df_clusters_est_err)} ({cluster_jump}th plotted)', transform=plt.gca().transAxes, fontsize = text_font_size)
    

    x_range = np.arange(low_lim, high_lim+1, 5)    
       
    param_y, cov_y = curve_fit(fit_func, df_clusters_est_err['vy[km/s]'], df_clusters_est_err['vy_est[km/s]'], 
                           sigma = df_clusters_est_err['vy_est_err[km/s]'], absolute_sigma=True)
    
    plt.text(x_pos_text, 0.09, f'Slope: {param_y[0].round(2)} $\pm$ {cov_y[0,0].round(2)}', transform=plt.gca().transAxes, fontsize = text_font_size)
    plt.text(x_pos_text, 0.03, f'Zero point: {param_y[1].round(2)} $\pm$ {cov_y[1,1].round(2)} km/s', transform=plt.gca().transAxes, fontsize = text_font_size)
    
    plt.plot(x_range, fit_func(x_range, *param_y), label = 'Linear fit', c = 'C2')
    plt.plot(x_range, x_range, label = '1-1 line', c = 'C1', alpha = 0.25, ls = 'dashed')
    plt.legend(loc = 'upper left')

    plt.subplot(3,2,4)
    residuals_y = fit_func(df_clusters_est_err['vy[km/s]'], *param_y) - df_clusters_est_err['vy_est[km/s]']
    plt.hist(residuals_y, bins = 100)
    plt.xlabel('Residual: V$_{\mathrm{y, fit}}$ - V$_{\mathrm{y, est}}$ (km/s)')
    plt.ylabel('Number of clusters')
    x_low, x_high = plt.xlim()
    plt.xlim(x_low, abs(x_low))
    y_low, y_high = plt.ylim()
    plt.ylim(y_low, y_high + 15)
    
    plt.text(0.02, 0.90, r'Mean (V$_{\mathrm{y, est}}$): ' + f"{round(np.mean(df_clusters_est_err['vy_est[km/s]']), 1)} km/s", transform=plt.gca().transAxes, fontsize = text_font_size)
    plt.text(0.02, 0.83, r'SD (V$_{\mathrm{y, est}}$): ' + f"{round(np.std(df_clusters_est_err['vy_est[km/s]']), 1)} km/s", transform=plt.gca().transAxes, fontsize = text_font_size)
    plt.text(0.02, 0.76, r'Mean (Error in V$_{\mathrm{y, est}}$): ' + f"{round(np.mean(df_clusters_est_err['vy_est_err[km/s]']), 1)} km/s", transform=plt.gca().transAxes, fontsize = text_font_size)
    plt.text(0.02, 0.69, r'SD (Error in V$_{\mathrm{y, est}}$): ' + f"{round(np.std(df_clusters_est_err['vy_est_err[km/s]']), 1)} km/s", transform=plt.gca().transAxes, fontsize = text_font_size)
    

    plt.text(x_pos_text_hist, 0.90, r'Cell size: {} h$^{}$kpc'.format(cell_size, minus_one), transform=plt.gca().transAxes, fontsize = text_font_size)
    plt.text(x_pos_text_hist, 0.83, r'Cube size: {} h$^{}$kpc'.format(clus_cube_size, minus_one), transform=plt.gca().transAxes, fontsize = text_font_size)
    mean_y = np.mean(residuals_y)
    plt.text(x_pos_text_hist, 0.76, f"Mean (residual): {round(mean_y, 1)} km/s", transform=plt.gca().transAxes, fontsize = text_font_size)
#     SD_y = np.sqrt(np.mean(residuals_y**2))
    SD_y = np.std(residuals_y)
    plt.text(x_pos_text_hist, 0.69, f"SD (residual): {round(SD_y, 1)} km/s", transform=plt.gca().transAxes, fontsize = text_font_size)
    r_vy = np.corrcoef(df_clusters_est_err['vy[km/s]'], df_clusters_est_err['vy_est[km/s]'])[1,0]
    plt.text(x_pos_text_hist, 0.62, f'Pearson\'s r: {round(r_vy, 2)}', transform=plt.gca().transAxes, fontsize = text_font_size)

    plt.title('Distribution of residuals in V$_{\mathrm{y, est}}$ fitting')




    plt.subplot(3,2,5)
    plt.errorbar(df_clusters_est_err['vz[km/s]'][::cluster_jump], df_clusters_est_err['vz_est[km/s]'][::cluster_jump], 
                 yerr = df_clusters_est_err['vz_est_err[km/s]'][::cluster_jump], fmt='.', ecolor='C6')
    plt.xlabel('V$_\mathrm{z, true}$ (km/s)')
    plt.ylabel('V$_\mathrm{z, estimated}$ (km/s)');
    plt.gca().set_xticks(range(-2000, 2001, 1000))
    plt.gca().set_yticks(range(-2000, 2001, 1000))
    plt.xlim(low_lim, high_lim)
    plt.ylim(-2000,2000)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.title('Vz - scatter plot')
    plt.text(x_pos_text, 0.15, f'Clusters: {len(df_clusters_est_err)} ({cluster_jump}th plotted)', transform=plt.gca().transAxes, fontsize = text_font_size)
    
    x_range = np.arange(low_lim, high_lim+1, 5)    
       
    param_z, cov_z = curve_fit(fit_func, df_clusters_est_err['vz[km/s]'], df_clusters_est_err['vz_est[km/s]'],
                           sigma = df_clusters_est_err['vz_est_err[km/s]'], absolute_sigma=True)
    
    plt.text(x_pos_text, 0.09, f'Slope: {param_z[0].round(2)} $\pm$ {cov_z[0,0].round(2)}', transform=plt.gca().transAxes, fontsize = text_font_size)
    plt.text(x_pos_text, 0.03, f'Zero point: {param_z[1].round(2)} $\pm$ {cov_z[1,1].round(2)} km/s', transform=plt.gca().transAxes, fontsize = text_font_size)
    
    plt.plot(x_range, fit_func(x_range, *param_z), label = 'Linear fit', c = 'C2')
    plt.plot(x_range, x_range, label = '1-1 line', c = 'C1', alpha = 0.25, ls = 'dashed')
    plt.legend(loc = 'upper left')

    plt.subplot(3,2,6)
    residuals_z = fit_func(df_clusters_est_err['vz[km/s]'], *param_z) - df_clusters_est_err['vz_est[km/s]']
    plt.hist(residuals_z, bins = 100)
    plt.xlabel('Residual: V$_{\mathrm{z, fit}}$ - V$_{\mathrm{z, est}}$ (km/s)')
    plt.ylabel('Number of clusters')
    x_low, x_high = plt.xlim()
    plt.xlim(x_low, abs(x_low))
    y_low, y_high = plt.ylim()
    plt.ylim(y_low, y_high + 15)
    
    plt.text(0.02, 0.90, r'Mean (V$_{\mathrm{z, est}}$): ' + f"{round(np.mean(df_clusters_est_err['vz_est[km/s]']), 1)} km/s", transform=plt.gca().transAxes, fontsize = text_font_size)
    plt.text(0.02, 0.83, r'SD (V$_{\mathrm{z, est}}$): ' + f"{round(np.std(df_clusters_est_err['vz_est[km/s]']), 1)} km/s", transform=plt.gca().transAxes, fontsize = text_font_size)
    plt.text(0.02, 0.76, r'Mean (Error in V$_{\mathrm{z, est}}$): ' + f"{round(np.mean(df_clusters_est_err['vz_est_err[km/s]']), 1)} km/s", transform=plt.gca().transAxes, fontsize = text_font_size)
    plt.text(0.02, 0.69, r'SD (Error in V$_{\mathrm{z, est}}$): ' + f"{round(np.std(df_clusters_est_err['vz_est_err[km/s]']), 1)} km/s", transform=plt.gca().transAxes, fontsize = text_font_size)
    

    plt.text(x_pos_text_hist, 0.90, r'Cell size: {} h$^{}$kpc'.format(cell_size, minus_one), transform=plt.gca().transAxes, fontsize = text_font_size)
    plt.text(x_pos_text_hist, 0.83, r'Cube size: {} h$^{}$kpc'.format(clus_cube_size, minus_one), transform=plt.gca().transAxes, fontsize = text_font_size)
    mean_z = np.mean(residuals_z)
    plt.text(x_pos_text_hist, 0.76, f"Mean (residual): {round(mean_z, 1)} km/s", transform=plt.gca().transAxes, fontsize = text_font_size)
#     SD_z = np.sqrt(np.mean(residuals_z**2))
    SD_z = np.std(residuals_z)
    plt.text(x_pos_text_hist, 0.69, f"SD (residual): {round(SD_z, 1)} km/s", transform=plt.gca().transAxes, fontsize = text_font_size)
    r_vz = np.corrcoef(df_clusters_est_err['vz[km/s]'], df_clusters_est_err['vz_est[km/s]'])[1,0]
    plt.text(x_pos_text_hist, 0.62, f'Pearson\'s r: {round(r_vz, 2)}', transform=plt.gca().transAxes, fontsize = text_font_size)

    plt.title('Distribution of residuals in V$_{\mathrm{z, est}}$ fitting')

    plt.subplots_adjust(top = 0.9, hspace = 0.4, wspace = 0.3)
    plt.tight_layout()
#     plt.show()
    plt.savefig(f'Plots/orig-sim-box/v_scatter_hist/{clus_cube_size_actual}/v_scatter_hist_{cell_size}.png')
    plt.close()
    clear_output(wait=False)
    
    return residuals_x, residuals_y, residuals_z, param_x[0], param_y[0], param_z[0]

**Examining the effects of cluster cube & cell sizes variation on velocity estimates**

In [ ]:
%%time
cell_size = 10000                      #h^-1 kpc, size of pixel or cell

cube_cell_size_assess = pd.DataFrame(columns=['Cell Size', 'Act Cube Size', 'Cube Size Set', 'Clusters', 'Mean - Vx', 'Mean - Vy', 
                                              'Mean - Vz', 'SD - Vx', 'SD - Vy', 'SD - Vz', 'm - Vx', 'm - Vy', 'm - Vz', 'r - Vx', 
                                              'r - Vy','r - Vz'])

cube_cell_size_assess_no_round = pd.DataFrame(columns=['Cell Size', 'Act Cube Size', 'Cube Size Set', 'Clusters', 'Mean - Vx', 'Mean - Vy', 
                                              'Mean - Vz', 'SD - Vx', 'SD - Vy', 'SD - Vz', 'm - Vx', 'm - Vy', 'm - Vz', 'r - Vx', 
                                              'r - Vy','r - Vz'])

# for clus_cube_size in [240000]:
#     for cell_size in [10000]:

for clus_cube_size in [160000, 200000, 240000, 280000, 320000]:
    for cell_size in [2000, 4000, 5000, 8000, 10000, 20000]:
        
        print(cell_size, clus_cube_size)
                
        no_of_cells = clus_cube_size//cell_size
        
        clus_cube_size_actual = clus_cube_size
        
        if no_of_cells % 2 != 0:
            clus_cube_size = clus_cube_size + cell_size
        
        sigma_in_pix = sigma_calc(cell_size)

        df_clusters = edge_clus_remover(clus_cube_size)

        delta_gal_mean = delta_gal_mean_func(cell_size)

        vel_terms = vel_terms_calc(cell_size, clus_cube_size)

        clus_param = list(zip(df_clusters['x[kpc/h]'], df_clusters['y[kpc/h]'], df_clusters['z[kpc/h]'], 
                              [delta_gal_mean]*len(df_clusters), [cell_size]*len(df_clusters), 
                              [vel_terms]*len(df_clusters), [sigma_in_pix]*len(df_clusters),
                              [clus_cube_size]*len(df_clusters)))

        pool = multi.Pool(processes = cores)
        v_est = pool.starmap(clus_velocity_calc, clus_param)

        df_clusters_est_err = df_clusters.copy()

        df_clusters_est_err['vx_est[km/s]'] = [i[0] for i in v_est]
        df_clusters_est_err['vy_est[km/s]'] = [i[1] for i in v_est]
        df_clusters_est_err['vz_est[km/s]'] = [i[2] for i in v_est]
        df_clusters_est_err['vx_est_err[km/s]'] = [i[3] for i in v_est]
        df_clusters_est_err['vy_est_err[km/s]'] = [i[4] for i in v_est]
        df_clusters_est_err['vz_est_err[km/s]'] = [i[5] for i in v_est]
        
        
        residuals_x, residuals_y, residuals_z, slope_x, slope_y, slope_z = plotting_func(df_clusters_est_err, cell_size, clus_cube_size_actual)

        cube_cell_size_assess = cube_cell_size_assess.append({'Cell Size': cell_size,
                'Act Cube Size': clus_cube_size_actual,
                'Cube Size Set': clus_cube_size,
                'Clusters': len(df_clusters),
                'Mean - Vx': round(np.mean(residuals_x),1), 
                'Mean - Vy': round(np.mean(residuals_y),1), 
                'Mean - Vz': round(np.mean(residuals_z),1), 
                'SD - Vx': round(np.std(residuals_x),1), 
                'SD - Vy': round(np.std(residuals_y),1),
                'SD - Vz': round(np.std(residuals_z),1), 
                'm - Vx': round(slope_x, 2),
                'm - Vy': round(slope_y, 2),
                'm - Vz': round(slope_z, 2),
                'r - Vx': round(np.corrcoef(df_clusters['vx[km/s]'], df_clusters_est_err['vx_est[km/s]'])[1,0],2),
                'r - Vy': round(np.corrcoef(df_clusters['vy[km/s]'], df_clusters_est_err['vy_est[km/s]'])[1,0],2),
                'r - Vz': round(np.corrcoef(df_clusters['vz[km/s]'], df_clusters_est_err['vz_est[km/s]'])[1,0],2)}, 
                ignore_index=True)
        
        cube_cell_size_assess_no_round = cube_cell_size_assess_no_round.append({'Cell Size': cell_size,
                'Act Cube Size': clus_cube_size_actual,
                'Cube Size Set': clus_cube_size,
                'Clusters': len(df_clusters),
                'Mean - Vx': np.mean(residuals_x), 
                'Mean - Vy': np.mean(residuals_y), 
                'Mean - Vz': np.mean(residuals_z), 
                'SD - Vx': np.std(residuals_x),
                'SD - Vy': np.std(residuals_y),
                'SD - Vz': np.std(residuals_z), 
                'm - Vx': slope_x,
                'm - Vy': slope_y,
                'm - Vz': slope_z,                                                                        
                'r - Vx': np.corrcoef(df_clusters['vx[km/s]'], df_clusters_est_err['vx_est[km/s]'])[1,0],
                'r - Vy': np.corrcoef(df_clusters['vy[km/s]'], df_clusters_est_err['vy_est[km/s]'])[1,0],
                'r - Vz': np.corrcoef(df_clusters['vz[km/s]'], df_clusters_est_err['vz_est[km/s]'])[1,0]}, 
                ignore_index=True)
        
        clus_cube_size = clus_cube_size_actual
        

2000 320000


In [51]:
print('Cell size: 10 Mpc, Cube size: 240 Mpc')
df_clusters_est_err.describe()

Cell size: 10 Mpc, Cube size: 240 Mpc


,x[kpc/h],y[kpc/h],z[kpc/h],m500c[Msol/h],vx[km/s],vy[km/s],vz[km/s],vx_est[km/s],vy_est[km/s],vz_est[km/s],vx_est_err[km/s],vy_est_err[km/s],vz_est_err[km/s]
count,1486.000000,1486.000000,1486.000000,1.486000e+03,1486.000000,1486.000000,1486.000000,1486.000000,1486.000000,1486.000000,1486.000000,1486.000000,1486.000000
mean,321167.437921,325547.000289,314873.843149,6.320741e+13,60.044705,55.125112,-9.400921,52.683729,53.477362,33.334068,127.899771,128.084933,127.802271
std,115128.818045,114647.361770,114015.385111,4.882252e+13,283.156687,296.834432,279.764800,255.516024,248.359802,260.344545,29.729645,29.755559,29.600705
min,120050.600000,121580.500000,120140.480000,3.163600e+13,-1179.530000,-1107.410000,-1064.920000,-969.476620,-912.710786,-895.401795,54.904741,54.521175,55.932559
25%,224259.405000,231868.690000,219766.045000,3.818072e+13,-114.970000,-134.777000,-190.498500,-104.346493,-107.421855,-127.576449,107.052365,106.957005,106.910924
50%,324557.625000,331498.735000,309916.205000,4.799315e+13,59.041100,69.447850,-16.468200,53.451941,57.379226,32.752925,127.044141,127.245839,127.355268
75%,419428.657500,425986.122500,415948.492500,7.034382e+13,237.558000,257.186750,176.384000,211.133283,219.125194,199.784999,146.468441,146.402916,146.013894
max,519955.530000,519124.750000,519470.590000,7.220310e+14,1109.160000,1164.960000,900.098000,993.867648,1083.820221,1053.365111,237.374550,235.400988,237.292442


In [54]:
cube_cell_size_assess

,Cell Size,Act Cube Size,Cube Size Set,Clusters,Mean - Vx,Mean - Vy,Mean - Vz,SD - Vx,SD - Vy,SD - Vz,m - Vx,m - Vy,m - Vz,r - Vx,r - Vy,r - Vz
0,2000.0,160000.0,160000.0,2473.0,-6.5,3.4,1.0,899.6,892.6,890.1,1.01,0.95,1.18,0.35,0.34,0.39
1,4000.0,160000.0,160000.0,2473.0,1.2,5.2,2.7,410.1,411.3,397.6,0.83,0.81,0.98,0.57,0.56,0.62
2,5000.0,160000.0,160000.0,2473.0,1.1,3.4,2.5,309.6,314.0,302.7,0.78,0.75,0.92,0.65,0.64,0.70
3,8000.0,160000.0,160000.0,2473.0,0.3,0.8,0.4,192.0,199.2,184.2,0.67,0.61,0.76,0.75,0.72,0.78
4,10000.0,160000.0,160000.0,2473.0,1.0,-0.4,-1.1,158.7,164.0,149.7,0.62,0.56,0.70,0.78,0.75,0.81
5,20000.0,160000.0,160000.0,2473.0,1.9,-0.1,0.5,112.2,108.4,106.5,0.47,0.41,0.51,0.79,0.76,0.80
6,2000.0,200000.0,200000.0,1928.0,-5.6,0.1,5.8,891.5,890.3,888.4,1.05,1.03,1.20,0.36,0.36,0.40
7,4000.0,200000.0,200000.0,1928.0,1.0,3.0,5.1,403.7,403.8,395.1,0.88,0.87,1.00,0.59,0.59,0.62
8,5000.0,200000.0,200000.0,1928.0,0.5,1.4,3.6,304.2,304.6,296.2,0.83,0.79,0.94,0.67,0.66,0.70
9,8000.0,200000.0,208000.0,1825.0,0.7,0.3,0.1,184.3,189.8,180.0,0.72,0.65,0.80,0.78,0.75,0.80


**Writing & seeing the assesment table**

In [55]:
cube_cell_size_assess_no_round.to_csv('Tables/orig-sim-box/cube_cell_size_assess_orig-sim-box_unrounded.tsv', index = False, sep = '\t')
cube_cell_size_assess.round(2).to_csv('Tables/orig-sim-box/cube_cell_size_assess_orig-sim-box_rounded.tsv', index = False, sep = '\t')
cube_cell_size_assess

,Cell Size,Act Cube Size,Cube Size Set,Clusters,Mean - Vx,Mean - Vy,Mean - Vz,SD - Vx,SD - Vy,SD - Vz,m - Vx,m - Vy,m - Vz,r - Vx,r - Vy,r - Vz
0,2000.0,160000.0,160000.0,2473.0,-6.5,3.4,1.0,899.6,892.6,890.1,1.01,0.95,1.18,0.35,0.34,0.39
1,4000.0,160000.0,160000.0,2473.0,1.2,5.2,2.7,410.1,411.3,397.6,0.83,0.81,0.98,0.57,0.56,0.62
2,5000.0,160000.0,160000.0,2473.0,1.1,3.4,2.5,309.6,314.0,302.7,0.78,0.75,0.92,0.65,0.64,0.70
3,8000.0,160000.0,160000.0,2473.0,0.3,0.8,0.4,192.0,199.2,184.2,0.67,0.61,0.76,0.75,0.72,0.78
4,10000.0,160000.0,160000.0,2473.0,1.0,-0.4,-1.1,158.7,164.0,149.7,0.62,0.56,0.70,0.78,0.75,0.81
5,20000.0,160000.0,160000.0,2473.0,1.9,-0.1,0.5,112.2,108.4,106.5,0.47,0.41,0.51,0.79,0.76,0.80
6,2000.0,200000.0,200000.0,1928.0,-5.6,0.1,5.8,891.5,890.3,888.4,1.05,1.03,1.20,0.36,0.36,0.40
7,4000.0,200000.0,200000.0,1928.0,1.0,3.0,5.1,403.7,403.8,395.1,0.88,0.87,1.00,0.59,0.59,0.62
8,5000.0,200000.0,200000.0,1928.0,0.5,1.4,3.6,304.2,304.6,296.2,0.83,0.79,0.94,0.67,0.66,0.70
9,8000.0,200000.0,208000.0,1825.0,0.7,0.3,0.1,184.3,189.8,180.0,0.72,0.65,0.80,0.78,0.75,0.80


**Reshaping the Pearson's r for Vz column into a more easy-to-read form**

In [56]:
r_Vz_table = pd.DataFrame()

for name, group in cube_cell_size_assess.groupby("Cell Size"):
    if r_Vz_table.empty:
        r_Vz_table = group.set_index("Act Cube Size")[["r - Vz"]].rename(columns={"r - Vz":name})
    else:
        r_Vz_table = r_Vz_table.join(group.set_index("Act Cube Size")[["r - Vz"]].rename(columns={"r - Vz":name}))

r_Vz_table

,2000.0,4000.0,5000.0,8000.0,10000.0,20000.0
Act Cube Size,,,,,,
160000.0,0.39,0.62,0.70,0.78,0.81,0.80
200000.0,0.40,0.62,0.70,0.80,0.82,0.82
240000.0,0.40,0.63,0.71,0.81,0.84,0.82
280000.0,0.41,0.64,0.72,0.82,0.85,0.84
320000.0,0.41,0.62,0.72,0.82,0.85,0.83


**Plotting the mean, SD & r response due to variation of cell sizes for a given cluster cube size**

In [58]:
x_axis = np.arange(0,6)

# for clus_cube_size in [240000]:
for clus_cube_size in [160000, 200000, 240000, 280000, 320000]:

    plt.figure(dpi = dpi, figsize = (5,9), facecolor=(1, 1, 1))
    plt.subplot(311)
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Act Cube Size'] == clus_cube_size]['Mean - Vx'], label = 'Vx')
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Act Cube Size'] == clus_cube_size]['Mean - Vy'], label = 'Vy')
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Act Cube Size'] == clus_cube_size]['Mean - Vz'], label = 'Vz')
    plt.legend()
    plt.xlabel(r'Cell size (h$^{-1}$kpc)')
    plt.ylabel('Mean of V$_{\mathrm{est}}$ (km/s)')
    plt.xticks(x_axis, [2000, 4000, 5000, 8000, 10000, 20000])
    minus_one = {-1}
    no_of_clus = int(cube_cell_size_assess[cube_cell_size_assess['Act Cube Size'] == clus_cube_size]['Clusters'].iloc[0])
    plt.title(r'Mean of velocity | Cube: {} h$^{}$Mpc | Clusters: {}'.format(int(clus_cube_size/1e3), minus_one, no_of_clus), fontsize = 10)

    plt.subplot(312)
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Act Cube Size'] == clus_cube_size]['SD - Vx'], label = 'Vx')
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Act Cube Size'] == clus_cube_size]['SD - Vy'], label = 'Vy')
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Act Cube Size'] == clus_cube_size]['SD - Vz'], label = 'Vz')
    plt.legend()
    plt.xlabel(r'Cell size (h$^{-1}$kpc)')
    plt.ylabel('SD of V$_{\mathrm{est}}$ (km/s)')
    plt.xticks(x_axis, [2000, 4000, 5000, 8000, 10000, 20000])
    minus_one = {-1}
    plt.title(r'SD of velocity | Cube: {} h$^{}$Mpc | Clusters: {}'.format(int(clus_cube_size/1e3), minus_one, no_of_clus), fontsize = 10)

    plt.subplot(313)
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Act Cube Size'] == clus_cube_size]['r - Vx'], label = 'Vx')
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Act Cube Size'] == clus_cube_size]['r - Vy'], label = 'Vy')
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Act Cube Size'] == clus_cube_size]['r - Vz'], label = 'Vz')
    plt.legend()
    plt.xlabel(r'Cell size (h$^{-1}$kpc)')
    plt.ylabel('Pearson\'s r')
    plt.xticks(x_axis, [2000, 4000, 5000, 8000, 10000, 20000])
    minus_one = {-1}
    plt.title(r"Pearson's r | Cube: {} h$^{}$Mpc | Clusters: {}".format(int(clus_cube_size/1e3), minus_one, no_of_clus), fontsize = 10)
    plt.subplots_adjust(hspace = 0.6)
    plt.tight_layout()
    plt.savefig(f'Plots/orig-sim-box/cell_size_exam/clus_box_{clus_cube_size}.png')
    plt.close()

**Plotting the mean, SD & r response due to variation of cluster cube sizes for a given cell size**

In [59]:

x_axis = np.arange(0,5)

# for cell_size in [5000]:
for cell_size in [2000, 4000, 5000, 8000, 10000, 20000]:

    plt.figure(dpi = dpi, figsize = (5,10), facecolor=(1, 1, 1))
    plt.subplot(311)
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Cell Size'] == cell_size]['Mean - Vx'], label = 'Vx')
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Cell Size'] == cell_size]['Mean - Vy'], label = 'Vy')
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Cell Size'] == cell_size]['Mean - Vz'], label = 'Vz')
    plt.legend()
    plt.xlabel(r'Cube size (h$^{-1}$kpc)')
    plt.ylabel('Mean of V$_{\mathrm{estimated}}$ (km/s)')
    plt.xticks(x_axis, [160000, 200000, 240000, 280000, 320000])
    plt.ylim(0,800)
    minus_one = {-1}
    plt.title(r'Mean of velocity | Cell size: {} h$^{}$kpc'.format(cell_size, minus_one), fontsize = 11)

    plt.subplot(312)
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Cell Size'] == cell_size]['SD - Vx'], label = 'Vx')
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Cell Size'] == cell_size]['SD - Vy'], label = 'Vy')
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Cell Size'] == cell_size]['SD - Vz'], label = 'Vz')
    plt.legend()
    plt.xlabel(r'Cube size (h$^{-1}$kpc)')
    plt.ylabel('SD of V$_{\mathrm{estimated}}$ (km/s)')
    plt.ylim(100, 1000)
    plt.xticks(x_axis, [160000, 200000, 240000, 280000, 320000])
    plt.title(r'SD of velocity | Cell size: {} h$^{}$kpc'.format(cell_size, minus_one), fontsize = 11)

    plt.subplot(313)
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Cell Size'] == cell_size]['r - Vx'], label = 'Vx')
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Cell Size'] == cell_size]['r - Vy'], label = 'Vy')
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Cell Size'] == cell_size]['r - Vz'], label = 'Vz')
    plt.legend()
    plt.xlabel(r'Cube size (h$^{-1}$kpc)')
    plt.ylabel('Pearson\'s r')
    plt.xticks(x_axis, [160000, 200000, 240000, 280000, 320000])
    plt.ylim(0.30, 0.90)
    plt.title(r"Pearson's r | Cell size: {} h$^{}$kpc".format(cell_size, minus_one), fontsize = 11)

    plt.subplots_adjust(hspace = 0.7)
    plt.tight_layout()
    plt.savefig(f'Plots/orig-sim-box/clus_bos_size_exam/cell_size_{cell_size}.png')
#     plt.savefig(f'Plots/orig-sim-box/clus_bos_size_exam/free_y_lim/cell_size_{cell_size}.png')
    plt.close()

**Finding the minimum value of mean & Sd, and maximum value of r**

In [ ]:
print(cube_cell_size_assess['Mean - Vx'].abs().min())
print(cube_cell_size_assess['Mean - Vy'].abs().min())
print(cube_cell_size_assess['Mean - Vz'].abs().min())
print(cube_cell_size_assess['SD - Vx'].abs().min())
print(cube_cell_size_assess['SD - Vy'].abs().min())
print(cube_cell_size_assess['SD - Vz'].abs().min())
print(cube_cell_size_assess['r - Vx'].max())
print(cube_cell_size_assess['r - Vy'].max())
print(cube_cell_size_assess['r - Vz'].max())

**Conclusions**
1. All three components of velocities behave similarly so anyone of them can be taken from now onwards as the line of sight velocity. Pearson's r is highest (0.85) for V$_\mathrm{z}$ so maybe prefer V$_\mathrm{z}$ .
2. The code to estimate velocity seems to work fine since:  
(a) mean error in V$_\mathrm{x}$ goes to 0.3 km/s for cell size of 10,000 h$^\mathrm{-1}$kpc & cluster box size of 160,000 h$^\mathrm{-1}$kpc (SD is 185 km/s & r is 	0.78 for this case)   
(b) standard deviation of error in V$_\mathrm{z}$ goes to 146 km/s for cell size of 10,000 h$^\mathrm{-1}$kpc & cluster box size of 320,000 h$^\mathrm{-1}$kpc (Mean is -41.7 km/s & r is 0.85 for this case)  
(c) Pearson's r goes to 0.85 for V$_\mathrm{z}$ estimates coming from cell size of 10,000 h$^\mathrm{-1}$kpc & cluster box size of 280,000 h$^\mathrm{-1}$kpc (Mean is -43 km/s & SD is 152 km/s for this case)
3. Cell size of 10,000 h$^\mathrm{-1}$kpc should be chosen since it gives least mean, SD & maximum r.
4. Cluster box size does not make much difference but higher values do give a litter better mean, SD and r. A value of 280,000 h$^\mathrm{-1}$kpc may be choszen since it give r of 0.85.